In [ ]:
from pymongo import MongoClient
import copy
from tqdm import tqdm

In [ ]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['YouTube_crawling']
collection = db['OneAsia_YouTube_crawling_copy1']
data_list_a = list(collection.find({"검색키워드" : "Busan One Asia Festival"}, {"제목" : 1, "URL" : 1, "채널명" : 1, "조회수" : 1, "업로드 날짜" : 1, "검색키워드" : 1, "설명" : 1, "좋아요" : 1, "댓글" : 1, "댓글 개수" : 1, "출연진" : 1, "공연일자" : 1}))    # 필요한 데이터의 내용
client.close()

In [ ]:
data_list1 = copy.deepcopy(data_list_a)

In [ ]:
# 업데이트 해줄 통합 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['OneAsia2024']
collection = db['youtube_oneasia_crawling_english']

In [ ]:
for month in ['2016.01', '2016.02', '2016.03', '2016.04', '2016.05', '2016.06', '2016.07', '2016.08', '2016.09', '2016.10', '2016.11', '2016.12',
        '2017.01', '2017.02', '2017.03', '2017.04', '2017.05', '2017.06', '2017.07', '2017.08', '2017.09', '2017.10', '2017.11', '2017.12',
        '2018.01', '2018.02', '2018.03', '2018.04', '2018.05', '2018.06', '2018.07', '2018.08', '2018.09', '2018.10', '2018.11', '2018.12',
        '2019.01', '2019.02', '2019.03', '2019.04', '2019.05', '2019.06', '2019.07', '2019.08', '2019.09', '2019.10', '2019.11', '2019.12',
        '2021.01', '2021.02', '2021.03', '2021.04', '2021.05', '2021.06', '2021.07', '2021.08', '2021.09', '2021.10', '2021.11', '2021.12',
        '2022.01', '2022.02', '2022.03', '2022.04', '2022.05', '2022.06', '2022.07', '2022.08', '2022.09', '2022.10', '2022.11', '2022.12',
        '2023.01', '2023.02', '2023.03', '2023.04', '2023.05', '2023.06', '2023.07', '2023.08', '2023.09', '2023.10', '2023.11', '2023.12']:
    # 초기 변수 설정
    new_data = {}       # 전체 데이터

    youtube_video = []     # 유튜브 데이터
    youtube_num = 0        # 유튜브 언급량 데이터
    cast_count = {}        # 출연진별 언급량 카운트

    # 언급량과 유튜브 데이터 생성
    for data in data_list1:
        if data['업로드 날짜'][:7] == month:  # 만약 월별이 아닌 연도별로 묶고싶으면 ['업로드 날짜'][:4]로 변경
            # 키워드별로 데이터를 찾는다.
            if 'Busan One Asia Festival' in data['검색키워드'] : #바꿔줘야할 검색키워드 부분
                youtube_data = {}
                youtube_data['제목'] = data['제목']
                youtube_data['채널명'] = data['채널명']
                youtube_data['URL'] = data['URL']
                youtube_data['조회수'] = data['조회수']
                youtube_data['업로드 날짜'] = data['업로드 날짜']
                youtube_data['좋아요'] = data['좋아요']
                youtube_data['설명'] = data['설명']
                youtube_data['댓글'] = data['댓글']
                youtube_data['댓글 개수'] = data['댓글 개수']
                youtube_data['공연일자'] = data['공연일자']
                
                # 출연진 정보가 있는 경우
                if '출연진' in data and data['출연진'] != "":
                    cast_list = data['출연진'].split(",") # 출연진을 쉼표로 나눠서 리스트로 변환
                    for cast in cast_list:
                        # 출연진별로 언급량 카운트
                        if cast.strip() in cast_count:
                            cast_count[cast.strip()] += 1
                        else:
                            cast_count[cast.strip()] = 1

                youtube_num += 1
                youtube_video.append(youtube_data)

    # 출연진별 언급량을 출연진목록 배열에 추가
    cast_list = []
    for cast, count in cast_count.items():
        cast_info = {
            "출연진": cast,
            "출연진언급량": count
        }
        cast_list.append(cast_info)

    new_data['_id'] = 'BusanOneAsiaFestival' + month
    new_data['기준년월'] = month                            
    new_data['유튜브'] = youtube_video                      
    new_data['키워드'] = 'BusanOneAsiaFestival' # 키워드 설정해줘야 함        
    new_data['유튜브언급량'] = youtube_num                        
    new_data['출연진목록'] = cast_list

    collection.insert_one(new_data)  

In [ ]:
client.close()